# Convolutional Neural Networks
In this tutorial we will see how to define, train and visualize a Convolutional Neural Network (CNN) for image classification using the [Keras](https://keras.io/) library.

## From theory...

Recall (see [here](http://cs231n.github.io/convolutional-networks/) for details):

> Convolutional Neural Networks are very similar to ordinary Neural Networks from the previous chapter: they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. So what does change? ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

Most CNNs have a feed-forward architecture and are tipically composed of [Convolutional](https://keras.io/layers/convolutional/), [Pooling](https://keras.io/layers/pooling/) and [Fully-Connected](https://keras.io/layers/core/#dense) layers.

CNN models first demonstrated their effectiveness for task of classification. In this context, a [Softmax](https://keras.io/activations/) activation after the last Dense layer encodes the probability distribution over the classes and the loss function employed is the categorical [cross-entropy](https://keras.io/objectives/).

More recently CNNs have been successfully employed to tackle a huge variety of problems, including e.g. semantic segmentation, visual attention, edge detection, image and video captioning etc. (see e.g. [here](https://github.com/kjw0612/awesome-deep-vision) for some examples). In order to deal with these different tasks, network architectures have evolved too from simple feed-forward to arbitrarily complex models.

## ...to practice.

[Keras](https://keras.io/) library provides facility for the [feed-forward](https://keras.io/models/sequential/) as well as for [more complex](https://keras.io/getting-started/functional-api-guide/) architectures. The latter can be used for defining composite models, such as multi-output models, directed acyclic graphs, or models with shared layers.

**In this tutorial, we train a simple feed-forward CNN to perform image classification** on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset, which consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. For this task, a [sequential](https://keras.io/models/sequential/) model is good enough: however, we'll tincker with the [functional API](https://keras.io/getting-started/functional-api-guide/) in order to acquire confidence with both syntaxes. Let's start!

First, we just load the CIFAR-10 data and normalize the training and test images to lie in the range $[0, 1]$. The only thing to note here is that for multi-class classification is often useful to convert the target from scalar values $t \in [0, 1, \dots, nb_{classes}]$ to *one-hot* vectors. For example, suppose we have $C=10$ classes: the target $t_i=3$, indicating that $x_i$ belongs to the class $3$, is converted to the following *one-hot* vector: $[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]$. Being this operation very common, Keras provides a built-in [function](https://keras.io/utils/np_utils/) to do this.

In [1]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.utils import np_utils

# cifar 10 classes
nb_classes = 10

# input image dimensions
img_rows, img_cols, img_channels = 32, 32, 3

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize X in range (0, 1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print('X_train shape:', X_train.shape)
print('Train samples: {}\nTest samples: {}.'.format(X_train.shape[0], X_test.shape[0]))

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Using TensorFlow backend.


X_train shape: (50000, 32, 32, 3)
Train samples: 50000
Test samples: 10000.


### Defining the model (sequential API)
If we follow the syntax of the sequential API, we need to create an instance of `Sequential()` and then stack all the layers one on the top of the other with `model.add(layer)`. Notice that for the first layer we have to specify the input's shape!

For `Convolution2D`, we have to specify the number of learned filters and the kernel size (written in explicit in the first layer). We can optionally specify other parameters, such as the `border_mode=same` not to change the width and height of the input. Remember that the default value `border_mode=valid` will reduce width and height by $k-1$, where $k$ is the size of the kernel. *ReLu* activations follow each convolutional layer and have no parameter.

Layers of `MaxPooling2D` take as parameter the height and width of the spatial `pool_size`. By default the pooling `strides` are set equal to the `pool_size`.

Layers of `Dropout` take a parameter $p \in [0,1]$, which represents the probability to set to 0 a certain input unit at each update during trainig time. As you should know, dropout is a form of regularization which helps prevent overfitting.

Finally, we have our fully-connected a.k.a. `Dense` layers: the first has 512 outputs, the second has as many outputs as the number of classes (10 in our case). The output of the second `Dense` layer is a vector containing `nb_classes` values in an arbitrary range. The `SoftMax` activation squashes these values to the range $[0,1]$, normalizing them such that they sum to $1$ (the math [here](https://en.wikipedia.org/wiki/Softmax_function)). 

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

model = Sequential()

model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

### Defining the model (functional API)
If we follow the functional API, things are slightly different. We have an `Input` instance which represents the input of the network. Each layer can now be applied just as a function to a certain input and produces a certain output. For example, the first `Convolution2D` is applied to `model_in` and produces the output `x`. Using this notation, appropriate layers can be stacked together to obtain the previous architecture. At the end, an instance of `Model` is created and is given the inputs and outputs of the computational graph.

In this simple case there's little difference with the sequential version: however, the functional syntax in general guarantees a huge flexibility in the construction of the network architecture. It's advisable to check the guide to the functional API [here](https://keras.io/getting-started/functional-api-guide/).

In [3]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten 
from keras.layers import Convolution2D, MaxPooling2D

model_in = Input(shape=X_train.shape[1:])

x = Convolution2D(32, 3, 3, activation='relu', border_mode='same') (model_in)
x = Convolution2D(32, 3, 3,  activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(nb_classes)(x)
model_out = Activation('softmax')(x)

model = Model(input=model_in, output=model_out)

### Everything is right? Doublecheck the summary!
A nice feature of Keras is that we can have access to a text summary of the whole model architecture by simply calling `model.summary()`

In [4]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 32, 32, 32)    896         input_1[0][0]                    
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 30, 30, 32)    9248        convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_6[0][0]            
___________________________________________________________________________________________

As we see, in the summary are reported all information about the computational graph that is going to be compiled. Make sure to understand both the output shape and the number of weights of each layer. Where do the most of the parameters come from?

In order to inspect the weights learned by each layer, we can also call the `layer.get_weights()` method. This will return a list with two elements, the first one being the weight tensor and the second one being the bias vector. Recall that convolutional layers learn a $(n_o, n_i, k, k)$ weight tensor, where $k$ is the size of the kernel, $n_i$ is the number of channels of the input tensor, and $n_o$ is the number of filters to be learned. A bias is also learned for each of the $n_o$ filters. Dense layers insted learn a $(n_i, n_o)$ weight tensor, where $n_o$ is the output size and $n_i$ is the input size of the layer. Each of the $n_o$ neurons also has a bias.

### Compiling the model

In order to compile the computational graph, you need to provide at least an optimizer and a loss function. In this case we employ as **optimizer** vanilla SGD with momentum: however, Keras provides a variety of different [bulti-in optimizers](https://keras.io/optimizers/), so feel free to try another one. Being the task multi-class classification, we unsurprisingly choose as **loss** function the [categorical cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy).

In [5]:
from keras.optimizers import SGD

# Let's train the model using SGD + momentum:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

### Let the training begin!

Now that everything is in place, we are ready to actually start the training of the network. First we set some parameters, in particular the **batch size** is the size of the batch used to perform the gradient descent. We can also decide to leverage on Keras [support](https://keras.io/preprocessing/image/) for **data augmentation**, which is always performed in CPU in a separate thread. In this case, we also apply some image transformation like rotations, shifts and flips to the training images, to reduce the risk of overfitting.

In [ ]:
# training parameters
batch_size = 128
nb_epoch = 200
data_augmentation = True

 Now we can really start. If everything has gone well the loss should gets down to 0.65 (test logloss) in 25 epochs, and down to 0.55 after 50 epochs.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/200
30848/50000 [=================>............] - ETA: 9s - loss: 2.3023 - acc: 0.1026